In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics, model_selection
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_excel('/content/drive/My Drive/SHRS_smallDS.xlsx',index_col=False)
print(data.shape)


(30000, 17)


In [4]:
missing_values = data.isnull().sum()

In [5]:
df_dropped = data.dropna()

In [ ]:
print(type('Survival months'))

<class 'str'>


In [6]:
data['Survival months'] = data['Survival months'].replace('Unknown', 0).astype(int)

In [7]:
# filter the dataset
def Survival(row):
    if (row['Survival months'] >= 60) and (row['Vital status recode (study cutoff used)'] == 'Alive'):
        return 1
    elif (row['Survival months'] < 60) and (row['SEER cause-specific death classification'] == 'Dead (attributable to this cancer dx)'):
        return 0
    else:
        data.drop(index=row.name, inplace=True)



data['Survived'] = data.apply(Survival, axis=1)
data[:5]

,Site recode ICD-O-3/WHO 2008,Behavior code ICD-O-3,Primary Site,Histologic Type ICD-O-3,Grade Recode (thru 2017),Marital status at diagnosis,Median household income inflation adj to 2021,Regional nodes examined (1988+),Regional nodes positive (1988+),SEER cause-specific death classification,Survival months,Vital status recode (study cutoff used),Age recode with <1 year olds,"Race recode (White, Black, Other)",Sex,Year of diagnosis,CS Tumor Size/Ext Eval (2004-2015),Survived
1,Rectum,Malignant,209,8140,Well differentiated; Grade I,Single (never married),"$75,000+",10,0,Dead (attributable to this cancer dx),49,Dead,65-69 years,White,Male,2000,Blank(s),0.0
2,Rectosigmoid Junction,Malignant,199,8140,Moderately differentiated; Grade II,Single (never married),"$75,000+",8,1,Alive or dead of other cause,155,Alive,35-39 years,White,Female,2008,3,1.0
3,Ascending Colon,Malignant,182,8140,Moderately differentiated; Grade II,Widowed,"$75,000+",14,1,Dead (attributable to this cancer dx),11,Dead,60-64 years,Black,Male,1995,Blank(s),0.0
4,Lung and Bronchus,Malignant,343,8012,Well differentiated; Grade I,Married (including common law),"$75,000+",1,1,Dead (attributable to this cancer dx),20,Dead,35-39 years,White,Female,1992,Blank(s),0.0
5,Breast,Malignant,506,8520,Moderately differentiated; Grade II,Divorced,"$75,000+",0,98,Dead (attributable to this cancer dx),14,Dead,75-79 years,White,Female,2003,Blank(s),0.0


In [8]:
print(type('CS Tumor Size/Ext Eval (2004-2015)'))

<class 'str'>


In [9]:
import re

#get the first two interger from a string
def get_first_two_ints(s):
    numbers = re.findall(r'\d+', s)
    return [int(num) for num in numbers[:2]]

# convert to numerical form
data['Age'] = None
data['Income'] = None
data['Tumor Size'] = None
data['Grade Recode'] = None

GR = {'Unknown': 0,'Undifferentiated; anaplastic; Grade IV': 4, 'Poorly differentiated; Grade III': 3, 'Moderately differentiated; Grade II': 2, 'Well differentiated; Grade I': 1}

for index, row in data.iterrows():
    data.at[index, 'Age'] = data.at[index, 'Age recode with <1 year olds'][:2]
    data.at[index, 'Income'] = get_first_two_ints(data.at[index, 'Median household income inflation adj to 2021'])[0]
    if data.at[index, 'CS Tumor Size/Ext Eval (2004-2015)'] == 'Blank(s)':
        data.at[index, 'Tumor Size'] = 0
    else:
        data.at[index, 'Tumor Size'] = data.at[index, 'CS Tumor Size/Ext Eval (2004-2015)']
    data.at[index, 'Grade Recode'] = GR[data.at[index, 'Grade Recode (thru 2017)']]

data['Survived'] = data['Survived'].astype(int)
data['Age'] = data['Age'].astype(int)
data['Income'] = data['Income'].astype(int)
data['Tumor Size'] = data['Tumor Size'].astype(int)
data['Grade Recode'] = data['Grade Recode'].astype(int)
data

,Site recode ICD-O-3/WHO 2008,Behavior code ICD-O-3,Primary Site,Histologic Type ICD-O-3,Grade Recode (thru 2017),Marital status at diagnosis,Median household income inflation adj to 2021,Regional nodes examined (1988+),Regional nodes positive (1988+),SEER cause-specific death classification,...,Age recode with <1 year olds,"Race recode (White, Black, Other)",Sex,Year of diagnosis,CS Tumor Size/Ext Eval (2004-2015),Survived,Age,Income,Tumor Size,Grade Recode
1,Rectum,Malignant,209,8140,Well differentiated; Grade I,Single (never married),"$75,000+",10,0,Dead (attributable to this cancer dx),...,65-69 years,White,Male,2000,Blank(s),0,65,75,0,1
2,Rectosigmoid Junction,Malignant,199,8140,Moderately differentiated; Grade II,Single (never married),"$75,000+",8,1,Alive or dead of other cause,...,35-39 years,White,Female,2008,3,1,35,75,3,2
3,Ascending Colon,Malignant,182,8140,Moderately differentiated; Grade II,Widowed,"$75,000+",14,1,Dead (attributable to this cancer dx),...,60-64 years,Black,Male,1995,Blank(s),0,60,75,0,2
4,Lung and Bronchus,Malignant,343,8012,Well differentiated; Grade I,Married (including common law),"$75,000+",1,1,Dead (attributable to this cancer dx),...,35-39 years,White,Female,1992,Blank(s),0,35,75,0,1
5,Breast,Malignant,506,8520,Moderately differentiated; Grade II,Divorced,"$75,000+",0,98,Dead (attributable to this cancer dx),...,75-79 years,White,Female,2003,Blank(s),0,75,75,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,Lung and Bronchus,Malignant,342,8041,Undifferentiated; anaplastic; Grade IV,Divorced,"$70,000 - $74,999",0,98,Dead (attributable to this cancer dx),...,55-59 years,White,Female,1991,Blank(s),0,55,70,0,4
29995,Gallbladder,Malignant,239,8140,Unknown,Divorced,"$70,000 - $74,999",0,98,Dead (attributable to this cancer dx),...,70-74 years,White,Female,1990,Blank(s),0,70,70,0,0
29997,Lung and Bronchus,Malignant,349,8140,Poorly differentiated; Grade III,Single (never married),"$70,000 - $74,999",0,98,Dead (attributable to this cancer dx),...,65-69 years,Black,Male,1991,Blank(s),0,65,70,0,3
29998,Lung and Bronchus,Malignant,341,8012,Unknown,Married (including common law),"$75,000+",0,98,Dead (attributable to this cancer dx),...,75-79 years,White,Male,1991,Blank(s),0,75,75,0,0


In [10]:
# Initialize a dictionary to store the mappings
category_mappings = {}

# Manually encode each categorical column to numerical values
for column in data.select_dtypes(include=['object']):
    # Create a mapping for the current column
    categories = data[column].unique()
    mapping = {category: index for index, category in enumerate(categories)}
    category_mappings[column] = mapping

    # Replace the categorical values with numerical values
    data[column] = data[column].replace(mapping)

print("Encoded DataFrame:")
print(data)

print("\nCategory Mappings:")
for column, mapping in category_mappings.items():
    print(f"{column}: {mapping}")

Encoded DataFrame:
       Site recode ICD-O-3/WHO 2008  Behavior code ICD-O-3  Primary Site  \
1                                 0                      0           209   
2                                 1                      0           199   
3                                 2                      0           182   
4                                 3                      0           343   
5                                 4                      0           506   
...                             ...                    ...           ...   
29993                             3                      0           342   
29995                             9                      0           239   
29997                             3                      0           349   
29998                             3                      0           341   
29999                             6                      0           549   

       Histologic Type ICD-O-3  Grade Recode (thru 2017)  \
1       

In [11]:
data

,Site recode ICD-O-3/WHO 2008,Behavior code ICD-O-3,Primary Site,Histologic Type ICD-O-3,Grade Recode (thru 2017),Marital status at diagnosis,Median household income inflation adj to 2021,Regional nodes examined (1988+),Regional nodes positive (1988+),SEER cause-specific death classification,...,Age recode with <1 year olds,"Race recode (White, Black, Other)",Sex,Year of diagnosis,CS Tumor Size/Ext Eval (2004-2015),Survived,Age,Income,Tumor Size,Grade Recode
1,0,0,209,8140,0,0,0,10,0,0,...,0,0,0,2000,0,0,65,75,0,1
2,1,0,199,8140,1,0,0,8,1,1,...,1,0,1,2008,1,1,35,75,3,2
3,2,0,182,8140,1,1,0,14,1,0,...,2,1,0,1995,0,0,60,75,0,2
4,3,0,343,8012,0,2,0,1,1,0,...,1,0,1,1992,0,0,35,75,0,1
5,4,0,506,8520,1,3,0,0,98,0,...,3,0,1,2003,0,0,75,75,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29993,3,0,342,8041,4,3,1,0,98,0,...,4,0,1,1991,0,0,55,70,0,4
29995,9,0,239,8140,2,3,1,0,98,0,...,7,0,1,1990,0,0,70,70,0,0
29997,3,0,349,8140,3,0,1,0,98,0,...,0,1,0,1991,0,0,65,70,0,3
29998,3,0,341,8012,2,2,0,0,98,0,...,3,0,0,1991,0,0,75,75,0,0


In [12]:
df1 = pd.DataFrame(data)

In [13]:
df1.to_excel('FairlearnMLdata.xlsx', sheet_name='Sheet1', index=False)
